## Projekt Briefkastenfirmen

## Imports

In [1]:
from selenium import webdriver
from selenium.webdriver.firefox.firefox_profile import FirefoxProfile # für den Tor-Browser
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary # für den Tor-Browser
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import re
from stem import Signal # Um beim Tor-Surfer die IP/Idendity zu wechseln
from stem.control import Controller #Um beim Tor-Surfer die IP/Idendity zu wechseln
from selenium.webdriver.support.ui import WebDriverWait # Um die Seite ganz zu laden brauchte ich aber doch nicht.
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common import exceptions 
from selenium.common.exceptions import ElementNotInteractableException
from random import randint #für random time sleep
from time import sleep #für random time sleep 
folderpath=r"C:\Users\Zoe\Desktop\maz-datenjournalismus\Daten_BR_Handelsreg"

# 1. Scrapen vom Handelsregister
- Ich will von allen 21000 Adressen des Kantons Zug die erste Page speichern. 
- Adressenaufbereitung: Siehe File **02 Adressen Aufbereitung**
#### Probleme beim Scrapen:
- die ersten 5000 gingen relativ gut. Danach ging fast gar nichts mehr.
#### Lösungsversuche:
- Kleinere Pakete schnüren und so immer wieder den Browser schliessen und neu öffnen.
- anstatt der clear-Funktion den clear Button drücken
- Browser wechseln
- Telefon an die zuständige IT
- Tor-Browser installieren
geholfen hat nichts davon
#### Lösung
- Exceptions um das ganze Gebilde einbauen
- Cookies bei jedem Browser-Neustart löschen 

## 1.1 Erste Seite aller Adressen abrufen und speichern
hier nun zuerst alle erste Seiten - so dass ich überall eine Anzahl habe. 
In einem weiteren Schritt könnte ich noch bei allen, die mehr als 20 Firmen haben (und damit mehr als eine Seite) die unterseiten durchgehen, um so an alle UIDs ranzukommen. Diese kann man dann via URL abrufen. Für meine erste Geschichte benötige ich diese aber nicht.
#### Dokumentation/Versuche unter 03 Scraping Handesregister/09 Tor-Scraping Handelsregister
- Die ersten rund 5000 konnte ich gut runterladen. Danach gingen immer weniger.
- Erst versuchte ich es mit 1000-er Päckchen nach welchen ich den Browser wieder neu öffnete. Ging nicht lange gut
- Dann die ersten Tor-Versuche: Versehentlich wechselte ich zuerst zu Firefox statt ganz zum Tor-Browser. Dies funktionierte besser als Chrome, aber auch nicht gut
- Dann fand ich den Dreh mit Tor heraus. Dies ging aber gar nicht mehr. 
- Danach die Lösung mit Chrome und den Exceptions. Auch hier konnte ich plötzlich nur noch 100-er Päckchen verarbeiten
-  Auch das Telefon mit der Handelsregister-IT brachte keine Lösung
- Finally: Mit Firefox arbeiten, ein Try/Except um das ganze und nach jedem Error das Fenster schiessen, Cookies löschen (wohl der wichtigste Teil der Sache) und wieder neu öffnen. So konnte ich die fehlenden rund 10'000 Files doch noch scrapen. 

#### Abflauf Scraping:
- Zuerst zog ich die ganze Stadt Zug runter. Notfalls hätte man es auch damit machen können. Denn gemäss dem Leiter des Handesregisters sind die relevanten Adressen wohl vorallem in der Stadt.

In [ ]:
dff=pd.read_csv(folderpath+"/BRFrames/alle_Adressen") #alle_Adressen: siehe 02 Adressen Aufbereitung

In [ ]:
counter=1000
nummer=21




#Wir starten den Browser auf
driver = webdriver.Firefox(executable_path='C:\geckodriver.exe') # hier mit dem Firefox (weil zuerst gedacht das sei schon der Tor)
binary = FirefoxBinary(r"C:\Users\Zoe\Desktop\Tor Browser\Browser\firefox.exe") 
#Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
time.sleep(randint(3,7)) #random time.sleep. In der Hoffnung dass er mich so weniger erkennt
#Hier drücke ich den Cookie-Banner weg
cookie=driver.find_element_by_id("j_idt198")
time.sleep(randint(3,7))            
cookie.click()#ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
sleep(randint(3,7))
# Hier klappe ich die Zusätzlichen Suchfelder auf
driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
time.sleep(3)

for adress in tqdm(dff["Adresse"]): #die Adressen durchgehen
        # Den Filter ansteuern, wo ich Domiziladresse auwählen kann
    try:
        su_fel_ad = driver.find_element_by_id('idSucheForm:panel:idDiverseTyp_label')
        time.sleep(randint(1,3))
        driver.execute_script('arguments[0].click();', su_fel_ad)
        time.sleep(randint(1,2))
        filt = driver.find_element_by_id("idSucheForm:panel:idDiverseTyp_filter")
        time.sleep(randint(1,3))
        filt.send_keys('Domiziladresse')
        filt.send_keys(Keys.ENTER)
        time.sleep(randint(1,4))

        search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
        search_adress.send_keys(adress) #Adresse Eingeben
        search_adress.send_keys(Keys.ENTER) #Enter drücken
        sleep(randint(5,10))
        page = driver.page_source.encode("utf-8") #Page in Variable speichern

        with open (folderpath+"/HRSeiten_Gemeinden/frontpage"+str(nummer)+"_"+str(counter)+".htm", "wb+") as file:
            file.write(page)
            file.close()

        pagename = "frontpage"+str(nummer)+"_"+str(counter)+".htm"        
        counter=counter+1 

         #Hier eine Try-Schlaufe weil auch das Clear nicht immer funktioniert. hat sich bewährt
        clear_all=driver.find_element_by_id("idSucheForm:j_idt167").click()
        time.sleep(randint(2,7))

    except StaleElementReferenceException:      
        print("musste neu starten - Stale")
         
        driver.close() # Hier das Browserfenster wieder schliessen, um danach erneut zu öffnen
        time.sleep(randint(10,300)) 
        driver = webdriver.Firefox(executable_path='C:\geckodriver.exe') # hier mit dem Firefox (weil zuerst gedacht das sei schon der Tor)
        binary = FirefoxBinary(r"C:\Users\Zoe\Desktop\Tor Browser\Browser\firefox.exe") 
        #Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
        driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
        time.sleep(randint(3,7)) #random time.sleep. In der Hoffnung dass er mich so weniger erkennt
        #Hier drücke ich den Cookie-Banner weg
        cookie=driver.find_element_by_id("j_idt198")
        time.sleep(randint(3,7))            
        cookie.click()#ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
        sleep(randint(3,7))
        # Hier klappe ich die Zusätzlichen Suchfelder auf
        driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
        time.sleep(3)
        

    except ElementNotInteractableException:
        print("musste neu starten - Element not")
       # Hier das Browserfenster wieder schliessen, um danach für das neue df erneut zu öffnen
        driver.close() # Hier das Browserfenster wieder schliessen, um danach erneut zu öffnen
        time.sleep(randint(10,300)) 
         # Hier das Browserfenster wieder schliessen, um danach für das neue df erneut zu öffnen
        driver = webdriver.Firefox(executable_path='C:\geckodriver.exe') # hier mit dem Firefox (weil zuerst gedacht das sei schon der Tor)
        binary = FirefoxBinary(r"C:\Users\Zoe\Desktop\Tor Browser\Browser\firefox.exe") 
        #Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
        driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
        time.sleep(randint(3,7)) #random time.sleep. In der Hoffnung dass er mich so weniger erkennt
        #Hier drücke ich den Cookie-Banner weg
        cookie=driver.find_element_by_id("j_idt198")
        time.sleep(randint(3,7))            
        cookie.click()#ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
        sleep(randint(3,7))
        # Hier klappe ich die Zusätzlichen Suchfelder auf
        driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
        time.sleep(3)
        
   
   

# 2. Nun Infos roh rausziehen
Roheinträge aus den gespeicherten Files abfragen
Danach mit Split in Pandas Infos rausziehen

In [ ]:
anzahl_liste=[]
path=folderpath+"/HRSeiten_Gemeinden/"
for page in tqdm(os.listdir(path)): #Ich erstelle direkt aus dem Folder eine Liste.
    file=open(path+page, 'r')
    page=page
    text=file.read()
    soup=BeautifulSoup(text, 'html.parser')
    div_list=soup.find_all("div")
    if len(div_list) == 97: #Wenn es keine Firma hat, hat es nur 97 Listen-Punkte
        anzahl= str("0") # Ich vermerke, dass es keine Firmen hat
        input_list=div_list[50].find_all("input") # Hier der Adressen-Eintrag
        adresse_o=input_list[0]
        adresse=str(adresse_o) #HIer noch ein Trick, dass er mir die Sache nicht als Liste ausspuckt
       
    else:
        anzahl= div_list[-26].text #Angabe mit Anzahl Firmen auslesen.
        input_list_f=div_list[-51].find_all("input") 
        adresse_f=input_list_f[0]
        adresse=str(adresse_f) # hier derselbe Trick wie oben
        

    minidict_a={"Seite":page,"Anzahl":anzahl, "Adresse":adresse}
    anzahl_liste.append(minidict_a)

In [ ]:
# Hier nun die rohe Liste mit den Adressen, Files und Anzahl Firmen.
df_adan=pd.DataFrame(anzahl_liste)

In [ ]:
df_adan[df_adan["Adresse"].notnull()==False]

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
df_adan.to_csv(folderpath+"/BRFrames/Alle_Gemeinden_ausgelesen_roh")# abspeichern

In [ ]:
#df_adan=pd.read_csv("BRFrames/Stadtzug_frontpages_ausgelesen_roh")

## 2.1 Zahl und Adresse extrahieren
mit Regex vom Rest trennen. 

In [ ]:
df_adan=pd.read_csv(folderpath+"/BRFrames/Alle_HR_frontpages_ausgelesen_roh")

In [ ]:
df_adan.shape #kommt hin, wird wohl jedoch noch 

In [ ]:
# Hier nun die Formel, um aus allen die Zahl rausziehen zu können
df_adan["Anzahl"]=df_adan["Anzahl"].str.extract("(\d+)")

In [ ]:
# und noch die Adresse
df_adan["Adresse"]=df_adan["Adresse"].str.extract("value=\"(.*)\"")

In [ ]:
df_adan

In [ ]:
df_adan.to_csv(folderpath+"/BRFrames/Alle_HR_extrahiert_unbereinigt")

## 2.2 Umlaut-Codes bereinigen
wichtig für den Abgleich mit der Adress-Liste. 
- So sehe ich, ob Adressen noch fehlen
- Kann so später die Dataframes wieder zusammenfügen für Geopandas

In [ ]:
df_adan=pd.read_csv(folderpath+"/BRFrames/Alle_HR_extrahiert_unbereinigt")

In [ ]:
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã¼","ü")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã¶","ö")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã¤","ä")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã„","Ä")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã–","Ö")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ãœ","Ü")
df_adan["Adresse"]=df_adan["Adresse"].str.replace("Ã©","é")

In [ ]:
df_adan.drop_duplicates("Adresse", inplace=True)

In [ ]:
df_adan.shape

In [ ]:
df_adan.to_csv(folderpath+"/BRFrames/alle_HR_extrahiert_umlaute_weg")

# 3. Abgleichen von Adress-Frame und Scrape-Frame
Sind alle Adressen runtergezogen 

In [ ]:
df_adr=pd.read_csv(folderpath+"/BRFrames/alle_adressen_nan_bereinigt") # die ursprünglichen Adressen 

In [ ]:
df_adr.shape

In [ ]:
# so zusammenführen dass auch die fehlenden Daten sichtbar werden
df_abgleich=pd.merge(df_adan, df_adr, on="Adresse", how="outer") 

In [ ]:
df_abgleich.drop_duplicates("Adresse", inplace=True)# Doppelte Einträge löschen

In [ ]:
df_abglnanweg=df_abgleich.dropna(subset=['GDENAME'])

In [ ]:
df_abglnanweg

In [ ]:
df_abglnanweg.shape, df_adr.shape #Kontrollieren ob es gleich viele sind wie bei der Stadt Zug

## 3.1 Wenn die Länge noch nicht stimmt und es noch "NaN"-Werte hat

#### Alle Adressen filtern, die NAN-Werte haben - also nicht funktionierten https://stackoverflow.com/questions/22551403/python-pandas-filtering-out-nan-from-a-data-selection-of-a-column-of-strings

In [ ]:
df_fehlende=df_abglnanweg[df_abglnanweg["Seite"].notnull()==False] 

In [ ]:
df_fehlende

In [ ]:
df_abgleich.to_csv(folderpath+"/BRFrames/alle_fehlende_adressem")

In [ ]:
df_abgleich[df_abgleich["DEINR"].notnull()==False].tail(50)

## 3.2 Wenns stimmt, alle Adressen da sind und das Frame bereinigt mit Ä und Ü:
### Abspeichern

In [ ]:
df_abgleich.to_csv(folderpath+"/BRFrames/Alle_Adressen_HR_sauber")

In [ ]:
df_sauber=df_abgleich

# 4. Scraping Telefonbuch
Nach dem ersten Visieren der Daten sowie dem Auffliegen der Luanda Leaks mit einem Akteur in Zug, wir mir bewusst, dass ich etwas benötige, mit dem ich die Grösse eines Gebäudes abschätzen kann. Leider ergaben die Recherchen bei Post sowie dem Kanton und Bund keine bessere Lösung als das Telefonbuch. Registriert werden nämlich nur Wohnungen. Geäude ohne Wohnungen sind gar nicht im Register. Das Telefonbuch soll also als Hilfe dienen, auch kleinere Hubs - wie das Luanda Leak, dass zu einem Haus in der Altstadt führt, in dem 40! Firmen registriert sind (normalerweise max. 3 Parteien pro Haus)- ausfindig zu machen. 

### Vorgehen
- Ich filtere all jene Adressen raus, bei denen überhaupt Firmen registriert sind.
- Diese werden dann beim Telefonbuch abgefragt. (So müssen nicht alle 21000 Adressen durchsucht werden)

In [ ]:
df_af=pd.read_csv(folderpath+"/BRFrames/        ")

In [ ]:
df_af["Anzahl"]=df_af["Anzahl"].astype("int")

In [ ]:
# Alle Adressen mit mindesrens einem Eintrag
df_af_s=df_af[df_af["Anzahl"]>0]

In [ ]:
counter=1

driver = webdriver.Firefox(executable_path='C:\geckodriver.exe') # jetzt via Tor mit unterschiedlichen IP's
binary = FirefoxBinary(r"C:\Users\Zoe\Desktop\Tor Browser\Browser\firefox.exe") 
#Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
driver.get("https://tel.search.ch/")
for adresse in tqdm(df_af_s["Adresse"]):
    eingabe=driver.find_element_by_name("wo")
    eingabe.send_keys(adresse) #Adresse Eingeben
    time.sleep(1)
    eingabe.send_keys(Keys.ENTER) #Enter drücken
    time.sleep(5)
    telpage = driver.page_source.encode("utf-8") #Page in Variable speichern

    with open (folderpath+"/Telbuch_Seiten/telfront"+str(counter)+".htm", "wb+") as file:
        file.write(telpage)
        file.close()

    telpagename = "telfront"+str(counter)+".htm"        
    counter=counter+1 

    clear_wo=driver.find_element_by_name("wo")
    clear_wo.clear()
    time.sleep(1)



minidicttelbuch={"Adresse":adresse, "Dokument":telpage}
listetelbuch.append(minidicttelbuch)

## 4.1 Auslesen der Seiten
Wie schon beim Handesregister nehme ich auch hier den Teil der Adresse als ganzes raus und extrahiere danach mit demselben Code die Adresse. Auch die Umlautbereinigung geht gleich.

In [ ]:
anzahl_liste_t=[]
path_t=folderpath+"/Telbuch_Seiten/"
for page_t in os.listdir(path_t):
    file_t=open(path_t+page_t,encoding="mbcs")
    text_t=file_t.read()
    soup_t=BeautifulSoup(text_t, 'html.parser')
    div_t_list=soup_t.find_all("div")
    anzahl_t=div_t_list[11].find_all("span")[0].text
    input_list=div_t_list[12].find_all("input")
    adresse_input=input_list[1]
    adresse_t=str(adresse_input)

    minidict_t={"Seite":page_t,"Anzahl":anzahl_t, "Adresse":adresse_t}
    anzahl_liste_t.append(minidict_t)

In [ ]:
dftel_t=pd.DataFrame(anzahl_liste_t) #in ein Dataframe
dftel_t.to_csv(folderpath+"\Telbook_alle_roh") # und speichern

In [ ]:
dftel_t["Adresse"]=dftel_t["Adresse"].str.extract("value=\"(.*)\"") #Adresse auslesen mit demselben Code wie HR

In [ ]:
dftel_t.to_csv(folderpath+"/BRFrames/Telbuch_alle_Adressen_extrahiert")

In [ ]:
dftel_t=pd.read_csv(folderpath+"/BRFrames/Telbuch_Stadt_Adressen_extrahiert")

In [ ]:
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ã¼","ü")
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ã¶","ö")
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ã¤","ä")
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ã„","Ä")
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ã–","Ö")
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ãœ","Ü")
dftel_t["Adresse"]=dftel_t["Adresse"].str.replace("Ã©","é")

In [ ]:
dftel_t.to_csv(folderpath+"/BRFrames/Telbuch_Stadt_Adressen_extrahiert_mit_richtigen_Umlauten")

# Handelsregister und Telbuch zusammenbringen
An der Adresse mergen. mal mit outer, damit ich sehe ob ich "überschuss" oder sonstige Ungereimtheiten habe.

In [ ]:
dftel_t=pd.read_csv(folderpath+"/BRFrames/Telbuch_Stadt_Adressen_extrahiert_mit_richtigen_Umlauten")

In [ ]:
df_sauber=pd.read_csv(folderpath+"/BRFrames/Stadt_Zug_sauber")

In [ ]:
df_ht=pd.merge(dftel_t, df_sauber, how="outer", on="Adresse")

In [ ]:
df_ht

In [ ]:
# Hats noch Nans? Bestens, nur eines... und dieses hat auch keine Firmen
df_ht[df_ht["Seite_x"].notnull()==False]

In [ ]:
# Wir wollen nun keine Nullen
df_ht1=df_ht[df_ht["Seite_x"].notnull()==True]
df_ht1.shape # sooo es stimmt wieder

In [ ]:
df_ht1.to_csv(folderpath+"/BRFrames/Stadt_sauber_HR_und_Telbuch")

# 4. Daten anschauen, hübschen, sortieren

**Fragen**
- Wo hat es am meisten Firmen?
- wieviele Adressen haben mehr als 20 Firmen (und müssten noch "nachgescrapet" werden)?

In [2]:
df_ht1=pd.read_csv(folderpath+"/BRFrames/Stadt_sauber_HR_und_Telbuch")

## 4.1 Klein machen und Zahlen in int umwandeln

In [20]:
# Ich brauche nur einen Teil
df_saukl=df_ht1[["Adresse","Anzahl_y","Seite_y","Anzahl_x","Seite_x","STRNAME","GDENAME","GKODE","GKODN"]]
df_saukl

,Adresse,Anzahl_y,Seite_y,Anzahl_x,Seite_x,STRNAME,GDENAME,GKODE,GKODN
0,"Aabachstrasse 1, Zug",0,frontpage0.htm,17,telfront1.htm,Aabachstrasse,Zug,2681243.278,1225319.466
1,"Aabachstrasse 19b, Zug",0,frontpage9.htm,2,telfront10.htm,Aabachstrasse,Zug,2681265.906,1225774.453
2,"Alte Baarerstrasse 3, Zug",0,frontpage99.htm,7,telfront100.htm,Alte Baarerstrasse,Zug,2682283.468,1225265.711
3,"Chamerstrasse 56, Zug",27,frontpage1037.htm,13,telfront1000.htm,Chamerstrasse,Zug,2680767.178,1225465.852
4,"Chamerstrasse 56.1, Zug",0,frontpage1040.htm,0,telfront1001.htm,Chamerstrasse,Zug,2680855.194,1225553.006
...,...,...,...,...,...,...,...,...,...
4685,"Chamerstrasse 51.z, Zug",0,frontpage1032.htm,1,telfront995.htm,Chamerstrasse,Zug,2680782.000,1225353.000
4686,"Chamerstrasse 52, Zug",11,frontpage1033.htm,5,telfront996.htm,Chamerstrasse,Zug,2680846.193,1225345.734
4687,"Chamerstrasse 53, Zug",0,frontpage1034.htm,1,telfront997.htm,Chamerstrasse,Zug,2680773.323,1225372.849
4688,"Chamerstrasse 54, Zug",3,frontpage1035.htm,4,telfront998.htm,Chamerstrasse,Zug,2680828.182,1225361.153


In [21]:
# Umbenennen der Columns-Namen
df_saukl.rename(columns={"Anzahl_y":"Firmen", "Anzahl_x":"Telbuch"}, inplace=True)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [22]:
df_saukl1=df_saukl[df_saukl["Telbuch"].str.contains("291'604")==False]# Es hat noch ein komischs Ergebnis
#An der Adresse gibt es aber gar keine Firma. Isch schmeisse diese dehalb gleich weg

In [23]:
# Um bei der anschliessenden Division keinen Fehler zu bekommen muss ich noch alle 0-er Telbucheinträge zu 1nern machen.
df_saukl1["Telbuch"]=df_saukl1["Telbuch"].str.replace("0","1")

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
df_saukl1["Firmen"]=df_saukl1["Firmen"].astype("int")


C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
df_saukl1["Telbuch"]=df_saukl1["Telbuch"].astype("int32")

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [26]:
df_saukl1.dtypes

Adresse     object
Firmen       int32
Seite_y     object
Telbuch      int32
Seite_x     object
STRNAME     object
GDENAME     object
GKODE      float64
GKODN      float64
dtype: object

## 4.2 Wertspalte für Auswertung: Firmen geteilt durch Telbuch
Im Zuge der Recherche/Datenerhebung wurde klar, dass einen Referenzgrösse nötig ist. So führten die Luanda Leaks in die Unter Altstadt, wo in einem Gebäude mit normalerweise drei Parteien 40 Firmen registriert waren. 
Im grössten Firmen-Hub sind wohl über 300 Firmen registriert (die 40 würden also nicht auffallen). Bei den 300 sind aber auch rund 30 Telefonbuch-Einträge zu finden. (Hat auch die Stichprobe für 1-2 anderen der grössten gezeigt.) Teilt man also die Zahl der Firmen durch die Telefonbucheinträge erhält man einen Wert, der eher als Referenz dienen könnte. 

In [27]:
df_saukl1["Vergleich"]=df_saukl1["Firmen"]/df_saukl1["Telbuch"]

C:\Users\Zoe\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [28]:
df_saukl1

,Adresse,Firmen,Seite_y,Telbuch,Seite_x,STRNAME,GDENAME,GKODE,GKODN,Vergleich
0,"Aabachstrasse 1, Zug",0,frontpage0.htm,17,telfront1.htm,Aabachstrasse,Zug,2681243.278,1225319.466,0.000000
1,"Aabachstrasse 19b, Zug",0,frontpage9.htm,2,telfront10.htm,Aabachstrasse,Zug,2681265.906,1225774.453,0.000000
2,"Alte Baarerstrasse 3, Zug",0,frontpage99.htm,7,telfront100.htm,Alte Baarerstrasse,Zug,2682283.468,1225265.711,0.000000
3,"Chamerstrasse 56, Zug",27,frontpage1037.htm,13,telfront1000.htm,Chamerstrasse,Zug,2680767.178,1225465.852,2.076923
4,"Chamerstrasse 56.1, Zug",0,frontpage1040.htm,1,telfront1001.htm,Chamerstrasse,Zug,2680855.194,1225553.006,0.000000
...,...,...,...,...,...,...,...,...,...,...
4685,"Chamerstrasse 51.z, Zug",0,frontpage1032.htm,1,telfront995.htm,Chamerstrasse,Zug,2680782.000,1225353.000,0.000000
4686,"Chamerstrasse 52, Zug",11,frontpage1033.htm,5,telfront996.htm,Chamerstrasse,Zug,2680846.193,1225345.734,2.200000
4687,"Chamerstrasse 53, Zug",0,frontpage1034.htm,1,telfront997.htm,Chamerstrasse,Zug,2680773.323,1225372.849,0.000000
4688,"Chamerstrasse 54, Zug",3,frontpage1035.htm,4,telfront998.htm,Chamerstrasse,Zug,2680828.182,1225361.153,0.750000


In [29]:
df_saukl1.to_csv(folderpath+"/BRFrames/Stadt_ganz_mit_Vergleichswert")

## 4.3 Sortieren  und anschauen

In [30]:
# Alle Adressen mit mindesrens einem Eintrag
df_saukl2=df_saukl1[df_saukl1["Vergleich"]>0]
df_saukl2

,Adresse,Firmen,Seite_y,Telbuch,Seite_x,STRNAME,GDENAME,GKODE,GKODN,Vergleich
3,"Chamerstrasse 56, Zug",27,frontpage1037.htm,13,telfront1000.htm,Chamerstrasse,Zug,2680767.178,1225465.852,2.076923
15,"Chamerstrasse 56a, Zug",4,frontpage1050.htm,1,telfront1011.htm,Chamerstrasse,Zug,2680843.426,1225394.913,4.000000
16,"Chamerstrasse 61, Zug",3,frontpage1051.htm,3,telfront1012.htm,Chamerstrasse,Zug,2680565.382,1225536.612,1.000000
21,"Chamerstrasse 63, Zug",4,frontpage1056.htm,1,telfront1017.htm,Chamerstrasse,Zug,2680545.495,1225542.482,4.000000
23,"Chamerstrasse 65, Zug",4,frontpage1058.htm,2,telfront1019.htm,Chamerstrasse,Zug,2680530.197,1225560.294,2.000000
...,...,...,...,...,...,...,...,...,...,...
4678,"Chamerstrasse 45, Zug",2,frontpage1026.htm,5,telfront989.htm,Chamerstrasse,Zug,2680878.372,1225294.140,0.400000
4680,"Chamerstrasse 47, Zug",1,frontpage1027.htm,4,telfront990.htm,Chamerstrasse,Zug,2680840.404,1225315.633,0.250000
4683,"Chamerstrasse 50, Zug",15,frontpage1030.htm,12,telfront993.htm,Chamerstrasse,Zug,2680869.425,1225328.661,1.250000
4686,"Chamerstrasse 52, Zug",11,frontpage1033.htm,5,telfront996.htm,Chamerstrasse,Zug,2680846.193,1225345.734,2.200000


In [31]:
df_saukl2.to_csv(folderpath+"/BRFrames/Stadt_Vergleich_grosser_als_Null")

In [16]:
# Hier die Topadressen. Wie vermutet grösstenteils an der Baarerstrasse.
df_saukl2.sort_values("Vergleich",ascending=False).head(40)

,Adresse,Firmen,Seite_y,Telbuch,Seite_x,STRNAME,Vergleich
3823,"Baarerstrasse 11.2, Zug",152,frontpage465.htm,1,telfront444.htm,Baarerstrasse,152.000000
117,"Dammstrasse 21, Zug",28,frontpage1147.htm,1,telfront1103.htm,Dammstrasse,28.000000
3259,"Unter Altstadt 15, Zug",19,frontpage3983.htm,1,telfront3932.htm,Unter Altstadt,19.000000
626,"Grafenauweg 10, Zug",99,frontpage1616.htm,6,telfront1562.htm,Grafenauweg,16.500000
4206,"Bahnhofstrasse 17, Zug",23,frontpage585.htm,2,telfront563.htm,Bahnhofstrasse,11.500000
4210,"Bahnhofstrasse 20, Zug",148,frontpage589.htm,13,telfront567.htm,Bahnhofstrasse,11.384615
717,"Guggitalring 12, Zug",11,frontpage1698.htm,1,telfront1644.htm,Guggitalring,11.000000
683,"Gubelstrasse 11, Zug",149,frontpage1667.htm,14,telfront1613.htm,Gubelstrasse,10.642857
55,"Chamerstrasse 77, Zug",31,frontpage1087.htm,3,telfront1048.htm,Chamerstrasse,10.333333
4211,"Bahnhofstrasse 21, Zug",328,frontpage590.htm,32,telfront568.htm,Bahnhofstrasse,10.250000


In [ ]:
# Anzahl der Adressen, die nochmals durchgegangen werden müssten - wenn ich die UIDs abfragen will
df_saukl[df_saukl["Anzahl"]>20]

In [ ]:
# Hotspots sind also tatsächlich die Baarersrasse, Bahnhofstrasse, Und weiter hinten ann Industriestrasse sowie Chamerstrasse 
df_saukl[df_saukl["Anzahl"]>20].groupby("STRNAME").count().sort_values("Anzahl", ascending=False)

In [ ]:
df_saukl[df_saukl["Adresse"].str.contains("Unter Altstadt")==True]